# scipy.integrate

### Adrian Price-Whelan

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from scipy import integrate

This subpackage contains integration schemes for both ordinary functions and differential equations. 

## Function integration

The `quad()` function (which uses Fortran's QUADPACK) provides a simple way to evaluate a definite integral over the specified domain. As a simple example, let's integrate the function
$$
f(x) = a x^3 + b x^2 + c
$$

from -5 to 5 for two choices of parameters. We start be defining a Python function to evaluate this function given the parameters:

In [ ]:
def f(x, a, b, c):
    return a*x**3 + b*x**2 + c

Now we'll integrate the function with
$$
a = 1.\\
b = 2.\\
c = -1.
$$

In [ ]:
val,err = integrate.quad(f, -5., 5, args=(1.,2.,-1.))
print(val)

The function returns the value of the integral and the error.

We can change the parameters of the function and re-compute the integral by changing the arguments:

In [ ]:
val,err = integrate.quad(f, -5., 5, args=(-3.,6.,0.))
print(val)

---

There are other integration schemes that operate on pre-computed grids of function values. For example, to use the trapezoidal rule or Simpson's rule on a fixed grid of points, we first have to call the function over some grid. We'll use the same function and domain as above but with 1024 samples:

In [ ]:
x = np.linspace(-5,5,1024)
y = f(x, 1., 2, -1)

val = integrate.trapz(y, x=x)
print(val)

This differs slightly from the value computed with QUADPACK. We could also try Simpson's rule:

In [ ]:
val = integrate.simps(y, x=x)
print(val)

There, that's a bit closer.

## ODE integration

This is a light wrapper around `lsoda` from Fortran's ODEPACK, which is a very fast library for integrating ordinary differential equations. Let's start with a simple example that has probably haunted you since you took the physics GRE: the double pendulum. ([Check here](http://scienceworld.wolfram.com/physics/DoublePendulum.html) if you want to follow along with the math, but I'm going to just start by writing down the Hamiltonian for the system assuming $l_1 = l_2 = m_1 = m_2 = 1.$ (in whatever units you need to make that work...)). 

In fact, to make things even simpler for us, I'm going to introduce a new Python library called Sympy (if you don't have it, you can `conda install sympy` or `pip install sympy`). Sympy is used for symbolic manipulation in Python -- think of it like bringing Mathematica functionality in to Python. One nice thing we can do is use it to take derivatives for us, then output code (so we don't make any mistakes coding things up).

I'll use the variables `p1` and `p2` to represent $p_{\theta_1}$ and $p_{\theta_2}$, and `t1` and `t2` for $\theta_1$ and $\theta_2$.

In [ ]:
import sympy as sym
sym.init_printing() # for pretty-printing Latex
from sympy.utilities import codegen # for generating code from equations

In [ ]:
p1,p2,t1,t2 = sym.symbols('p1, p2, t1, t2')
g = sym.symbols('g') # gravitational acceleration

In [ ]:
H = (p1**2 + 2*p2**2 - 2*p1*p2*sym.cos(t1-t2)) / (2 * (1 + sym.sin(t1-t2)**2)) - g*sym.cos(t2) - 2*g*sym.cos(t1)
H

We can double check that we coded up the Hamiltonian correctly using the nicely formatted Latex representation of the equation. The derivatives we need are $\dot{\theta}_1$, $\dot{\theta}_2$, $\dot{p_{\theta_1}}$, and $\dot{p_{\theta_2}}$:

In [ ]:
# create python functions to evaluate Hamilton's equations:
t1_dot = sym.lambdify((t1,t2,p1,p2,g), sym.diff(H, p1), modules='numpy')
t2_dot = sym.lambdify((t1,t2,p1,p2,g), sym.diff(H, p2), modules='numpy')
p1_dot = sym.lambdify((t1,t2,p1,p2,g), -sym.diff(H, t1), modules='numpy')
p2_dot = sym.lambdify((t1,t2,p1,p2,g), -sym.diff(H, t2), modules='numpy')

Now we'll create a function to actually form the system of ODEs that we will pass in to the ODE solver in scipy:

In [ ]:
def derivs(w, t, g):
    """ 
    Evaluate the derivatives of w = (theta1, theta2, p_theta1, p_theta2), 
    e.g., Hamilton's equations, at time t.
    """
    dH = np.zeros_like(w)
    
    args = list(w) + [g]
    dH[0] = t1_dot(*args)
    dH[1] = t2_dot(*args)
    dH[2] = p1_dot(*args)
    dH[3] = p2_dot(*args)
    
    return dH

In [ ]:
# create a time array from 0 to 100 with 1024 steps
t = np.linspace(0., 20, 1024)

# t1 and t2 are the initial angles (degrees)
# p1 and p2 are the initial angular velocities (degrees per second)
t1 = 120.0
t2 = -10.0
p1 = 0.0
p2 = 0.0

# initial state -- convert from degrees (above) to radians
state = np.radians([t1, t2, p1, p2])

# integrate your ODE using scipy.integrate.
y = integrate.odeint(derivs, state, t, args=(9.8,))

Now we convert the solution to cartesian coordinates to plot the trajectory:

In [ ]:
x1 = np.sin(y[:,0])
y1 = -np.cos(y[:,0])

x2 = np.sin(y[:,1]) + x1
y2 = -np.cos(y[:,1]) + y1

In [ ]:
plt.figure(figsize=(6,6))
plt.plot(x1, y1, marker=None)
plt.plot(x2, y2, marker=None)
plt.xlim(-1,1)
plt.ylim(-2.5,0.5)
plt.xlabel("x")
plt.ylabel("y")

---

<h1 style='background-color: #cccccc; padding: 15px;'>Exercises</h1>

We're going to estimate the number of RR Lyrae stars in a wedge out of a spherical shell centered on the Sun. Let's assume the number density of RR Lyrae stars has a density profile of the form:
$$
\rho(r) = \ \rho_0 \, \left(\frac{r_0}{r}\right)^\alpha
$$

with $\rho_0 = 4.5~{\rm kpc}^{-3}$, $r_0 = 8~{\rm kpc}$, $\alpha = 2.5$, and $r = \sqrt{x^2+y^2+z^2}$ where $x,y,z$ are defined relative to the Galactic center.

This density relation is defined relative to the Galactic center. Start by writing a function to compute the number density given a Galactic longitude, latitude, and _heliocentric_ distance.

In [ ]:
def density(l, b, d):
    x = d*np.cos(l)*np.cos(b) - 8.
    y = d*np.sin(l)*np.cos(b)
    z = d*np.sin(b)
    r = np.sqrt(x**2 + y**2 + z**2)
    
    # TODO: evaluate the density relation at r
    # return val

2) Integrate this density over a portion of a spherical shell centered on the Sun:
$$
100^\circ < l < 160^\circ\\
-35^\circ < b < -15^\circ\\
10~{\rm kpc} < d < 20~{\rm kpc}
$$

You'll first need to write a function to evaluate the integrand:
$$
\iiint \rho(l,b,d) \, d^2 \, \cos b \, {\rm d}b \, {\rm d}l \, {\rm d}d
$$

In [ ]:
def integrand(l, b, d):
    # TODO: evaluate integrand
    pass

3) Use `integrate.tplquad` to integrate the integrand -- how many RR Lyrae stars are expected in this region?